# TRABAJO FIN DE MASTER - DATOS TRÁFICO
# **Author**: Cristóbal León-Salas
# **Date**: 2025-08-02

# LIBRERIAS

Se cargan las siguietnes librerias:

-  pandas --> Para tablas de datos.
-  numpy --> para cálculos numéricos y para trabajar con matrices y vectores.
-  os --> Para trabajar con directorios, archivos, carpetas,...
-  matplotlib --> Para hacer visualizaciones gráficas básicas.
-  seaborn --> Para gráficos estadísticos más profesionales y de fácil interpretación.
-  warnings --> Para evitar mensajes de advertencias
-  product -->  Para sacar todas las combinaciones posibles entre los elementos de dos o más listas
-  folium –> Para representación georreferenciada


In [1]:
import pandas as pd
import logging, numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from itertools import product
from IPython.display import display
import folium
import logging

# Ignorar el SettingWithCopyWarning
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)
sns.set()

# FUNCIONES

## FUNCIÓN "clean_excel_sheets"

La función "clean_excel_sheets" me va a permitir eliminar aquellas hojas de los Excels que no contienen la información que necesito.  

In [2]:
def clean_excel_sheets(filepath):
    # Cargo todas las hojas del archivo
    xls = pd.ExcelFile(filepath)
    sheet_names = xls.sheet_names
    
    # Filtro hojas que NO sean las que queremos eliminar
    sheets_to_keep = [sheet for sheet in sheet_names if sheet not in ["ESTACIONES", "UBICACIÓN ESTACIONES"]]

    # Leo solo las hojas que queremos mantener
    df_dict = {sheet: xls.parse(sheet) for sheet in sheets_to_keep}

    return df_dict

## FUNCIÓN asignar_zona

Esta funcion me va a permitir clasificar las estaciones en una zona u otra

In [3]:
def asignar_zona(lat, lon):
    if lat >= lat_60:
        zona_ns = 'Norte'
    elif lat <= lat_30:
        zona_ns = 'Sur'
    else:
        zona_ns = ''

    if lon <= lon_30:
        zona_ew = 'Oeste'
    elif lon >= lon_60:
        zona_ew = 'Este'
    else:
        zona_ew = ''

    if zona_ns and zona_ew:
        return zona_ns + zona_ew  # Ej: 'Noreste', 'Suroeste'
    elif zona_ns:
        return zona_ns
    elif zona_ew:
        return zona_ew
    else:
        return 'Centro'

## FUNCIÓN extract_digits_to_int

Función para normalizar los códigos de las estaciones de tráfico

In [4]:
def extract_digits_to_int(series):
   # Quito todo lo que no sean dígitos y convierte a entero (Int64) preservando NaN.
    s = series.astype(str).str.replace(r"\D+", "", regex=True)  # dejo solo dígitos
    s = s.replace("", pd.NA)                                    # vacío -> NA
    return pd.to_numeric(s, errors="coerce").astype("Int64")

## FUNCIÓN fourier_series

Función que va a construir  las variables explicativas periódicas (seno y coseno) para usarlas en la regreseion.

In [5]:
def fourier_series(dates: pd.DatetimeIndex, period: float, order: int) -> np.ndarray:
    
    """
    Genera componentes de la serie de Fourier (senos y cosenos) hasta el armónico 'order'
    para un conjunto de fechas dado y un período específico.

    Parámetros:
    - dates: índice de fechas (pd.DatetimeIndex). Una secuencia de fechas
    - period: período base (por ejemplo, 365 para anual)
    - order: número de armónicos a generar

    Retorna:
    - Matriz numpy con columnas de senos y cosenos para cada armónico
    """

    # Si el orden es 0 o negativo, no se genera ninguna componente
    if order <= 0:
        return np.empty((len(dates), 0))  # Devuelve una matriz vacía con tantas filas como fechas

    # Convierte las fechas en días desde la primera fecha
    t = (dates - dates[0]).days.values.astype(float)

    # Lista para almacenar las componentes seno y coseno
    X = []

    # Itera sobre cada armónico desde 1 hasta 'order'
    for k in range(1, order + 1):
        # Agrega la componente seno para el armónico k
        X.append(np.sin(2 * np.pi * k * t / period))
        # Agrega la componente coseno para el armónico k
        X.append(np.cos(2 * np.pi * k * t / period))

    # Combina todas las componentes en una sola matriz (columnas = características)
    return np.column_stack(X) # Devuelve una matriz con tantas columnas como funciones seno/coseno se hayan pedido.

## FUNCIÓN design_fourier

Esta función sirve para construir una matriz de diseño que se puede usar como base explicativa en modelos de regresión para series temporales.

In [6]:
def design_fourier(dates: pd.DatetimeIndex, include_trend=True,
                   yearly_order=3, weekly_order=3) -> np.ndarray:

    # include_trend: si es True, se incluye una variable de tendencia lineal.
    # yearly_order: número de armónicos para la estacionalidad anual. SE PUEDE CAMBIAR PARA AJUSTAR MODELO.
    # weekly_order: número de armónicos para la estacionalidad semanal. SE PUEDE CAMBIAR PARA AJUSTAR MODELO.
    """Intercepto + (tendencia lineal) + Fourier anual + Fourier semanal."""
    n = len(dates)
    parts = [np.ones(n)] # Columna de unos para representar el término constante
    if include_trend:
        t = (dates - dates[0]).days.values.astype(float) # Resultado: un array como [0.0, 1.0, 2.0, ..., n] que representa el tiempo transcurrido desde la primera fecha.
        parts.append(t) # Esto añade esa columna de tendencia a la lista parts, que se usará para construir la matriz final
    Xy = fourier_series(dates, period=365.25, order=yearly_order)
    Xw = fourier_series(dates, period=7.0,    order=weekly_order)
    if Xy.size: parts.append(Xy) # Si no está vacía se añade a parts.
    if Xw.size: parts.append(Xw) # Si no está vacía se añade a parts.
    return np.column_stack(parts)

## FUNCIÓN fit_predict_fourier

Esta función realiza un ajuste de regresión lineal usando un diseño determinista basado en componentes de Fourier, y luego genera predicciones para un conjunto de fechas futuro o pasado

In [7]:
def fit_predict_fourier(dates_train: pd.DatetimeIndex, y_train: np.ndarray,
                        dates_pred: pd.DatetimeIndex) -> np.ndarray:
    """Ajuste lineal con diseño determinista (Fourier) y predicción sobre dates_pred."""
    X_tr = design_fourier(dates_train, include_trend=True, yearly_order=3, weekly_order=3)
    beta = np.linalg.pinv(X_tr) @ y_train
    # Ajusta el modelo por mínimos cuadrados.
    # np.linalg.pinv(X_tr) calcula la pseudoinversa de la matriz de diseño. Una herramienta matemática que permite resolver sistemas de ecuaciones lineales
    # @ y_train aplica el producto matricial → obtiene los coeficientes del modelo (beta).
    
    X_te = design_fourier(dates_pred,  include_trend=True, yearly_order=3, weekly_order=3) # Construye la matriz de diseño para las fechas de predicción.
    return X_te @ beta # Aplica el modelo ajustado (beta) sobre las fechas de predicción. Devuelve los valores estimados para dates_pred.

## FUNCIÓN _import_prophet

Esta función sirve para importar el modelo Prophet

In [8]:
def _import_prophet():
    try:
        from prophet import Prophet  # paquete moderno
        return Prophet
    except Exception:
        try:
            from fbprophet import Prophet  # fallback legacy
            return Prophet
        except Exception as e:
            raise ImportError(
                "No se pudo importar Prophet. Instala con: pip install prophet (o fbprophet)."
            ) from e

Prophet = _import_prophet()

## FUNCIÓN prophet_fit_predict_full

Esta función utiliza el modelo Prophet para ajustar una serie temporal observada y generar predicciones sobre un rango completo de fechas. Aporta una mayo flexibilidad gracias a los splines internos de Prophet que aportan tendencia

In [9]:
def prophet_fit_predict_full(dates_obs, y_obs, full_index, mcmc_samples=0, seed=42):
    df_tr = pd.DataFrame({"ds": dates_obs, "y": y_obs})
    m = Prophet(
        growth="linear",
        yearly_seasonality=True,
        weekly_seasonality=True,
        daily_seasonality=False,
        seasonality_mode="additive",
        mcmc_samples=mcmc_samples,   # si >0, activa MCMC
    )
    # Semilla de aleatoriedad
    m.fit(df_tr, seed=seed)

    df_future = pd.DataFrame({"ds": full_index})
    yhat_full = m.predict(df_future)["yhat"].values
    return pd.Series(yhat_full, index=full_index, name=f"{VALUE_COL}_EXT")

## FUNCIÓN rmse

Calcula el error cuadrático medio entre dos vectores: y_true (valores reales) y y_pred (valores predichos).

In [10]:
def rmse(y_true, y_pred):
    check = np.isfinite(y_true) & np.isfinite(y_pred) # Esto evita errores si hay valores faltantes o inválidos.
    return np.sqrt(np.mean((y_true[check] - y_pred[check])**2)) if check.any() else np.nan

## FUNCIÓN mae

Calcula el Mean Absolute Error (MAE), o error absoluto medio, entre dos vectores: y_true (valores reales) y y_pred (valores predichos).

In [11]:
def mae(y_true, y_pred):
    check = np.isfinite(y_true) & np.isfinite(y_pred)
    return np.mean(np.abs(y_true[check] - y_pred[check])) if check.any() else np.nan

## FUNCIÓN rolling_cv_zone

Esta función realiza una validación cruzada temporal (rolling-origin cross-validation) para comparar dos modelos de predicción de series temporales

In [12]:
def rolling_cv_zone(dates_obs: pd.DatetimeIndex,
                    y_obs: np.ndarray,
                    horizon: int = 15,
                    step: int = 15,
                    min_train: int = 90,
                    max_splits: int = 20) -> pd.DataFrame:
    """
    Devuelve un DataFrame con columnas:
    ['split','model','rmse','mae','train_start','train_end','test_start','test_end'].
    """
    # Sanitizar entradas
    assert isinstance(dates_obs, pd.DatetimeIndex), "dates_obs debe ser DatetimeIndex"
    order = np.argsort(dates_obs.values)
    dates_obs = dates_obs[order]
    y_obs = np.asarray(y_obs, dtype=float)[order]
    mask = np.isfinite(y_obs)
    dates_obs, y_obs = dates_obs[mask], y_obs[mask]

    n = len(y_obs)
    if n < (min_train + horizon):
        # Demasiado corta para al menos 1 split
        return pd.DataFrame(columns=["split","model","rmse","mae",
                                     "train_start","train_end","test_start","test_end"])

    end_train = max(min_train, int(0.5 * n))  # arranque razonable si la serie es larga
    split = 0
    rows = []

    while end_train + horizon <= n and split < max_splits:
        tr = slice(0, end_train)
        te = slice(end_train, end_train + horizon)

        dates_tr, y_tr = dates_obs[tr], y_obs[tr]
        dates_te, y_te = dates_obs[te], y_obs[te]

        # --- Fourier (función ya definida fit_predict_fourier) ---
        yhat_four = fit_predict_fourier(dates_tr, y_tr, dates_te)
        rows.append({
            "split": split, "model": "fourier",
            "rmse": rmse(y_te, yhat_four), "mae": mae(y_te, yhat_four),
            "train_start": dates_tr.min(), "train_end": dates_tr.max(),
            "test_start": dates_te.min(),  "test_end": dates_te.max()
        })

        # --- Prophet (MAP) ---
        yhat_prop = prophet_fit_predict_full(dates_tr, y_tr, dates_te,
                                        mcmc_samples=0, seed=42)
        rows.append({
            "split": split, "model": "prophet",
            "rmse": rmse(y_te, yhat_prop), "mae": mae(y_te, yhat_prop),
            "train_start": dates_tr.min(), "train_end": dates_tr.max(),
            "test_start": dates_te.min(),  "test_end": dates_te.max()
        })

        end_train += step
        split += 1

    return pd.DataFrame(rows)

## FUNCIÓN pick_winner

Esta función elige el mejor modelo para una zona específica

In [13]:
def pick_winner(df_zone, rel_tol=0.05):
    # df_zone: filas de una única ZONA con columnas ['model','rmse_mean','rmse_std','mae_mean','mae_std','n_splits']
    dfz = df_zone.dropna(subset=["rmse_mean"]).copy()
    if dfz.empty:
        return pd.Series({"best_model_by_rmse": None, "rmse_mean": np.nan, "mae_mean": np.nan,
                          "n_splits": int(df_zone["n_splits"].max() if not df_zone.empty else 0),
                          "ensemble_suggested": False})
    # Orden por RMSE, luego MAE
    dfz = dfz.sort_values(["rmse_mean", "mae_mean"], ascending=[True, True]).reset_index(drop=True)
    best = dfz.iloc[0] # El primer modelo (best) es el que tiene el menor error.
    if len(dfz) > 1:
        second = dfz.iloc[1]
        # diferencia relativa de RMSE
        if best["rmse_mean"] > 0:
            rel_diff = (second["rmse_mean"] - best["rmse_mean"]) / best["rmse_mean"]
        else:
            rel_diff = np.inf
        ensemble_flag = bool(rel_diff <= rel_tol)
    else:
        ensemble_flag = False

    return pd.Series({
        "best_model_by_rmse": best["model"],
        "rmse_mean": float(best["rmse_mean"]),
        "mae_mean": float(best["mae_mean"]),
        "n_splits": int(best["n_splits"]),
        "ensemble_suggested": ensemble_flag
    })


# DATOS DE TRÁFICO

## LECTURA DE DATOS DE TRÁFICO

Leo todos los archivos excels que contienen los datos del histórico de densidad de tráfico en Madrid

Leo los datos de tráfico obtenidos de esta web: https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=33cb30c367e78410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

In [14]:
df_total = pd.read_parquet('98_03_DatosTráfico.parquet', engine='pyarrow')

In [15]:
df_total.head()

,FDIA,FEST,FSEN,HOR1,HOR2,HOR3,HOR4,HOR5,HOR6,HOR7,HOR8,HOR9,HOR10,HOR11,HOR12,Unnamed: 15
0,2024-06-01,ES01,1-,900.0,640.0,457.0,399.0,386.0,406.0,300.0,515.0,715.0,818.0,992.0,1232.0,NaN
1,2024-06-01,ES01,1=,1602.0,1637.0,1126.0,1162.0,1350.0,1616.0,1678.0,1715.0,1383.0,1072.0,980.0,1193.0,NaN
2,2024-06-01,ES01,2-,762.0,592.0,406.0,357.0,322.0,383.0,312.0,355.0,612.0,969.0,1176.0,1335.0,NaN
3,2024-06-01,ES01,2=,1264.0,1098.0,868.0,957.0,1019.0,1088.0,977.0,1025.0,1063.0,852.0,779.0,914.0,NaN
4,2024-06-01,ES02,1-,375.0,375.0,341.0,303.0,268.0,226.0,186.0,170.0,239.0,274.0,400.0,384.0,NaN


In [16]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466808 entries, 0 to 466807
Data columns (total 16 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   FDIA         466808 non-null  datetime64[ns]
 1   FEST         466808 non-null  category      
 2   FSEN         466808 non-null  object        
 3   HOR1         466807 non-null  float64       
 4   HOR2         466808 non-null  float64       
 5   HOR3         466808 non-null  float64       
 6   HOR4         466808 non-null  float64       
 7   HOR5         466808 non-null  float64       
 8   HOR6         466808 non-null  float64       
 9   HOR7         466808 non-null  float64       
 10  HOR8         466808 non-null  float64       
 11  HOR9         466808 non-null  float64       
 12  HOR10        466808 non-null  float64       
 13  HOR11        466808 non-null  float64       
 14  HOR12        466808 non-null  float64       
 15  Unnamed: 15  1 non-null       floa

Elimino la variable "rara"

In [17]:
df_total = df_total.drop('Unnamed: 15', axis=1)

In [18]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466808 entries, 0 to 466807
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   FDIA    466808 non-null  datetime64[ns]
 1   FEST    466808 non-null  category      
 2   FSEN    466808 non-null  object        
 3   HOR1    466807 non-null  float64       
 4   HOR2    466808 non-null  float64       
 5   HOR3    466808 non-null  float64       
 6   HOR4    466808 non-null  float64       
 7   HOR5    466808 non-null  float64       
 8   HOR6    466808 non-null  float64       
 9   HOR7    466808 non-null  float64       
 10  HOR8    466808 non-null  float64       
 11  HOR9    466808 non-null  float64       
 12  HOR10   466808 non-null  float64       
 13  HOR11   466808 non-null  float64       
 14  HOR12   466808 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(12), object(1)
memory usage: 50.3+ MB


## IMPUTACIÓN VALORES AUSENTES

Muestro el número de valores ausentes de cada variable

In [19]:
tabla_nas = df_total.isna().sum().reset_index()
tabla_nas.columns = ['Variable', 'N_NAs']
tabla_nas['%_NAs'] = (tabla_nas['N_NAs'] / len(df_total) * 100).round(2)

tabla_nas = tabla_nas.sort_values(by='N_NAs', ascending=False)
display (tabla_nas)

,Variable,N_NAs,%_NAs
3,HOR1,1,0.0
1,FEST,0,0.0
0,FDIA,0,0.0
2,FSEN,0,0.0
4,HOR2,0,0.0
5,HOR3,0,0.0
6,HOR4,0,0.0
7,HOR5,0,0.0
8,HOR6,0,0.0
9,HOR7,0,0.0


Solo se observa un valor ausente. Dada la poca relevancia, lo estimo como el valor que se toma en HOR2 en esa misma instancia, de modo que:

In [20]:
# Solo imputar donde HOR1 es NaN
df_total.loc[df_total['HOR1'].isna(), 'HOR1'] = df_total.loc[df_total['HOR1'].isna(), 'HOR2']

Comprobación:

In [21]:
tabla_nas = df_total.isna().sum().reset_index()
tabla_nas.columns = ['Variable', 'N_NAs']
tabla_nas['%_NAs'] = (tabla_nas['N_NAs'] / len(df_total) * 100).round(2)

tabla_nas = tabla_nas.sort_values(by='N_NAs', ascending=False)
display (tabla_nas)

,Variable,N_NAs,%_NAs
0,FDIA,0,0.0
1,FEST,0,0.0
2,FSEN,0,0.0
3,HOR1,0,0.0
4,HOR2,0,0.0
5,HOR3,0,0.0
6,HOR4,0,0.0
7,HOR5,0,0.0
8,HOR6,0,0.0
9,HOR7,0,0.0


## AGRUPACIÓN DATOS POR ESTACIÓN Y DIA

Se crea una nueva variable TOTAL_HOR sumando las columnas HOR1 a HOR12

In [22]:

df_total["TOTAL_HOR"] = df_total.loc[:, "HOR1":"HOR12"].sum(axis=1)

#    - df.loc[:, 'HOR1':'HOR12'] selecciona todas las filas y columnas desde HOR1 hasta HOR12 (inclusive)
#    - .sum(axis=1) calcula la suma por fila (axis=1 → horizontal)

# Ordenar por FDIA de más tardía a más reciente
df_total = df_total.sort_values(by="FDIA", ascending=True).reset_index(drop=True)


In [23]:
df_total.head()

,FDIA,FEST,FSEN,HOR1,HOR2,HOR3,HOR4,HOR5,HOR6,HOR7,HOR8,HOR9,HOR10,HOR11,HOR12,TOTAL_HOR
0,2019-11-01,ES45,2=,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2019-11-01,ES39,2-,1055.0,0.0,279.0,219.0,157.0,120.0,128.0,296.0,435.0,723.0,916.0,1260.0,5588.0
2,2019-11-01,ES39,2=,1445.0,1464.0,1369.0,919.0,689.0,809.0,1111.0,894.0,878.0,803.0,699.0,509.0,11589.0
3,2019-11-01,ES40,1-,727.0,653.0,516.0,382.0,361.0,383.0,422.0,534.0,722.0,1075.0,1273.0,1526.0,8574.0
4,2019-11-01,ES40,1=,1726.0,1698.0,996.0,1120.0,1574.0,1796.0,1663.0,1502.0,1092.0,778.0,576.0,300.0,14821.0


Se comprueba que variable TOTAL_HOR ha sido calculada correctamente

Procedo ahora a eliminar las variables desde HOR1 a HOR12 y la variable FSEN, puesto que no considero que aporte un gran valor a nuestro estudio

In [24]:
df_total = df_total.drop(columns=df_total.loc[:, "HOR1":"HOR12"].columns)
df_total = df_total.drop(columns=["FSEN"])

In [25]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466808 entries, 0 to 466807
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   FDIA       466808 non-null  datetime64[ns]
 1   FEST       466808 non-null  category      
 2   TOTAL_HOR  466808 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(1)
memory usage: 7.6 MB


Paso la variable FDIA a temporal y la variable FEST a variables categóricas

In [26]:
# Paso FDIA a tipo fecha (día primero)
df_total["FDIA"] = pd.to_datetime(df_total["FDIA"], dayfirst=True, errors="coerce")

# Paso FEST a tipo categórico
df_total["FEST"] = df_total["FEST"].astype("category")

Finalmente, se procede a la agrupación de las instancias que tienen los mismos valores de FDIA y FEST

In [27]:

df_grouped = df_total.groupby(["FDIA", "FEST"], observed=True).agg({"TOTAL_HOR": "sum"}).reset_index()

# Agrupar por FDIA y FEST, sumando TOTAL_HOR
#    - groupby(["FDIA", "FEST"], observed=True) agrupa según esas dos columnas. Solo los datos observados.
#    - .agg({"TOTAL_HOR": "sum"}) indica que queremos sumar TOTAL_HOR en cada grupo
#    - .reset_index() devuelve un DataFrame limpio con índice secuencial


COMPROBACIÓN: Muestro los valores para un día aleatorio, ejemplo: 01/06/2024 y para una estacion, por ejemplo la ES01 para verificar que se ha agrupado todo correctamente

In [28]:
filtro_fecha = "2024-06-01"
filtro_estacion = "ES01"

valores_grouped = df_grouped[
    (df_grouped["FDIA"] == filtro_fecha) & (df_grouped["FEST"] == filtro_estacion)
]["TOTAL_HOR"]

valores_total = df_total[
    (df_total["FDIA"] == filtro_fecha) & (df_total["FEST"] == filtro_estacion)
]["TOTAL_HOR"]

# Muestro resultados
print("DataFrame agrupado:")
print(valores_grouped)

print("\nDataFrame original sin HOR1-HOR12:")
print(valores_total)


DataFrame agrupado:
98766    43759.0
Name: TOTAL_HOR, dtype: float64

DataFrame original sin HOR1-HOR12:
395064     7760.0
395214    16514.0
395215     7581.0
395216    11904.0
Name: TOTAL_HOR, dtype: float64


Coincide el primer valor con la suma de los valores sacado del dataframe original, por lo que la agrupación es correcta, por tanto:

In [29]:
df_total = df_grouped

In [30]:
df_total.head()

,FDIA,FEST,TOTAL_HOR
0,2019-11-01,ES01,38883.0
1,2019-11-01,ES02,33144.0
2,2019-11-01,ES03,33512.0
3,2019-11-01,ES04,20397.0
4,2019-11-01,ES05,54960.0


In [31]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116702 entries, 0 to 116701
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   FDIA       116702 non-null  datetime64[ns]
 1   FEST       116702 non-null  category      
 2   TOTAL_HOR  116702 non-null  float64       
dtypes: category(1), datetime64[ns](1), float64(1)
memory usage: 1.9 MB


In [32]:
# Paso TOTAL_HOR a tipo entero
df_total["TOTAL_HOR"] = df_total["TOTAL_HOR"].astype("int")

In [33]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116702 entries, 0 to 116701
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   FDIA       116702 non-null  datetime64[ns]
 1   FEST       116702 non-null  category      
 2   TOTAL_HOR  116702 non-null  int64         
dtypes: category(1), datetime64[ns](1), int64(1)
memory usage: 1.9 MB


## IMPUTACIÓN VALORES NULOS DE DATOS DE TRÁFICO

In [34]:
df_cero = df_total[df_total["TOTAL_HOR"] == 0]

In [35]:
df_cero.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167 entries, 69 to 71725
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   FDIA       167 non-null    datetime64[ns]
 1   FEST       167 non-null    category      
 2   TOTAL_HOR  167 non-null    int64         
dtypes: category(1), datetime64[ns](1), int64(1)
memory usage: 6.6 KB


Se observan algunas instancias donde el valor es 0. Se observa en que estaciones este valor es cero

In [36]:
# Filtrar las filas con TOTAL_HOR = 0 y extraer años únicos
estac_cero = (
    df_total.loc[df_total["TOTAL_HOR"] == 0, "FEST"]
    .dropna()            # eliminar posibles NaN
    .cat.remove_unused_categories()  # quitar categorías no usadas
    .unique()
)

print(estac_cero)

['ES11', 'ES20', 'ES39', 'ES09', 'ES08', 'ES34', 'ES33', 'ES47', 'ES42']
Categories (9, object): ['ES08', 'ES09', 'ES11', 'ES20', ..., 'ES34', 'ES39', 'ES42', 'ES47']


In [37]:
# Lista de categorías a eliminar
categorias_a_eliminar = ['ES11', 'ES20', 'ES39', 'ES09', 'ES08', 'ES34', 'ES33', 'ES47', 'ES42']

# Filtrar para quitar esas categorías
df_total = df_total[~df_total["FEST"].isin(categorias_a_eliminar)].copy()

# Quitar categorías no usadas de la variable categórica
df_total["FEST"] = df_total["FEST"].cat.remove_unused_categories()

In [38]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98900 entries, 0 to 116701
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   FDIA       98900 non-null  datetime64[ns]
 1   FEST       98900 non-null  category      
 2   TOTAL_HOR  98900 non-null  int64         
dtypes: category(1), datetime64[ns](1), int64(1)
memory usage: 2.4 MB


# DATOS ESTACIONES TRÁFICO

## GEORREFERENCIACIÓN ESTACIONES TRÁFICO

### LECTURA DATOS ESTACIONES

Leo los datos de las estaciones por medio de un archivo .xlsx

In [39]:
df_estaciones_trafico = pd.read_excel("95_25.08.08_DATOS_ESTACIONES.xlsx")

In [40]:
df_estaciones_trafico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Nº        60 non-null     int64  
 1   ESTACIÓN  60 non-null     object 
 2   LATITUD   60 non-null     float64
 3   LONGITUD  60 non-null     float64
dtypes: float64(2), int64(1), object(1)
memory usage: 2.0+ KB


In [41]:
df_estaciones_trafico.head()

,Nº,ESTACIÓN,LATITUD,LONGITUD
0,1,Paseo de la Castellana,40.431927,-3.689109
1,2,Calle Princesa,40.426068,-3.712709
2,3,Calle Doctor Esquerdo,40.417576,-3.669262
3,4,Paseo de San Franciso de Sales,40.440372,-3.716227
4,5,Paseo de Santa María de la Cabeza,40.406569,-3.694644


Paso la variable ESTACIÓN a categórica:

In [42]:
# Paso variables a categóricas:
df_estaciones_trafico['ESTACIÓN'] = df_estaciones_trafico['ESTACIÓN'].astype('category')

In [43]:
df_estaciones_trafico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Nº        60 non-null     int64   
 1   ESTACIÓN  60 non-null     category
 2   LATITUD   60 non-null     float64 
 3   LONGITUD  60 non-null     float64 
dtypes: category(1), float64(2), int64(1)
memory usage: 4.1 KB


In [44]:
categorias_a_eliminar = [11, 20, 39, 9, 8, 34, 33, 47, 42]  # enteros

df_estaciones_trafico_ = (
    df_estaciones_trafico[~df_estaciones_trafico["Nº"].isin(categorias_a_eliminar)]
    .copy()
)

df_estaciones_trafico_["Nº"] = (
    df_estaciones_trafico_["Nº"].astype("category").cat.remove_unused_categories()
)

In [45]:
# filas eliminadas
eliminadas = df_estaciones_trafico.shape[0] - df_estaciones_trafico_.shape[0]
print("Filas eliminadas:", eliminadas)

# categorías actuales
print(df_estaciones_trafico_["Nº"].cat.categories.tolist())

Filas eliminadas: 9
[1, 2, 3, 4, 5, 6, 7, 10, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 35, 36, 37, 38, 40, 41, 43, 44, 45, 46, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]


In [46]:
df_estaciones_trafico = df_estaciones_trafico_

In [47]:
df_estaciones_trafico_.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, 0 to 59
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Nº        51 non-null     category
 1   ESTACIÓN  51 non-null     category
 2   LATITUD   51 non-null     float64 
 3   LONGITUD  51 non-null     float64 
dtypes: category(2), float64(2)
memory usage: 4.2 KB


Creo un mapa ubicando cada una de las estaciones:

In [48]:
# Creo mapa centrado tomando la media de todas las latitudes y longitudes:

centro = [df_estaciones_trafico['LATITUD'].mean(), df_estaciones_trafico['LONGITUD'].mean()]
m = folium.Map(location=centro, zoom_start=12, tiles='Esri.WorldImagery')  # ortofoto

# Añado chinchetas para cada estación
for _, row in df_estaciones_trafico.iterrows():
    folium.Marker(
        location=[row['LATITUD'], row['LONGITUD']],
        popup=f"Estación {row['Nº']}",
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

# Guardo mapa como archivo HTML interactivo
m.save("03_25.08.08_mapa_estaciones_trafico.html")

# Muestro el mapa
display(m)

### DIVISIÓN ESTACIONES POR ZONAS

Para empezar, importo el criterio de latitudes y longitudes que delimitaban cada zona en las estaciones de calidad del aire

In [49]:
df_lat_long = pd.read_csv("02_25.08.07_percentiles_lat_lon.csv")

In [50]:
lat_30 = df_lat_long.loc[df_lat_long["PERCENTIL"] == "lat_30", "VALOR"].iloc[0]
lat_60 = df_lat_long.loc[df_lat_long["PERCENTIL"] == "lat_60", "VALOR"].iloc[0]
lon_30 = df_lat_long.loc[df_lat_long["PERCENTIL"] == "lon_30", "VALOR"].iloc[0]
lon_60 = df_lat_long.loc[df_lat_long["PERCENTIL"] == "lon_60", "VALOR"].iloc[0]

Se asingan las zonas a cada una de las estaciones

In [51]:
# Asignar zona al DataFrame
df_estaciones_trafico['ZONA'] = df_estaciones_trafico.apply(
    lambda row: asignar_zona(row['LATITUD'], row['LONGITUD']), axis=1
)

Represento cada estación en función de la zona asingada

In [52]:

# Renombrar las zonas compuestas a formato estándar
df_estaciones_trafico['ZONA'] = df_estaciones_trafico['ZONA'].replace({
    'NorteEste': 'Noreste',
    'NorteOeste': 'Noroeste',
    'SurEste': 'Sureste',
    'SurOeste': 'Suroeste'
})

# Asignamos un color distinto a cada zona
colores_zonas = {
    'NorteEste': 'blue',
    'Norte': 'darkblue',
    'Noreste': 'cadetblue',
    'Oeste': 'orange',
    'Centro': 'red',
    'Este': 'lightred',
    'Suroeste': 'purple',
    'Sur': 'green',
    'Sureste': 'lightgreen',
    'Sin clasificar': 'gray'
}

# Añadir chinchetas coloreadas por zona
for _, row in df_estaciones_trafico.iterrows(): # Es una forma muy común en pandas para iterar fila por fila en un DataFrame.
    # df_estaciones_filtrado.iterrows() --> Método de pandas que devuelve un generador que produce pares (índice, fila) del DataFrame, uno a uno.
    # for _, row in ... --> Se hace un bucle sobre esos pares.
    # _ Es una convención para decir "no me interesa el índice" (aunque está disponible si lo necesitas).
    # row --> Es una Serie de pandas que representa una fila del DataFrame, con los nombres de columnas como índice.
    zona = row['ZONA']
    color = colores_zonas.get(zona, 'gray')  # default: gris si zona no conocida
    
    folium.Marker(
        location=[row['LATITUD'], row['LONGITUD']],
        popup=f"Estación {row['Nº']}<br>ZONA: {zona}",
        icon=folium.Icon(color=color, icon='info-sign')
    ).add_to(m)

# Mostrar el mapa en Jupyter
display(m)

# Guardar también como HTML si quieres
m.save("03_25.08.08_mapa_estaciones_trafico_coloreado_por_zona.html")

In [53]:
df_estaciones_trafico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, 0 to 59
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   Nº        51 non-null     category
 1   ESTACIÓN  51 non-null     category
 2   LATITUD   51 non-null     float64 
 3   LONGITUD  51 non-null     float64 
 4   ZONA      51 non-null     object  
dtypes: category(2), float64(2), object(1)
memory usage: 4.6+ KB


Paso la variable ZONA a categórica:

In [54]:
df_estaciones_trafico['ZONA'] = df_estaciones_trafico['ZONA'].astype('category')

Limpio el dataframe eliminando y renombrando algunas de las variables:

In [55]:
# 1. Elimino la variable "ESTACIÓN"
df_estaciones_trafico = df_estaciones_trafico.drop(columns=["ESTACIÓN"])

# 2. Cambio el nombre de la variable Nº por "ESTACION"
df_estaciones_trafico = df_estaciones_trafico.rename(columns={"Nº": "ESTACION"})

# 3. Elimino las variables "LATITUD" y "LONGITUD"
for col in ["LATITUD", "LONGITUD"]:
    if col in df_estaciones_trafico.columns:
        df_estaciones_trafico = df_estaciones_trafico.drop(columns=[col])

In [56]:
df_estaciones_trafico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, 0 to 59
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   ESTACION  51 non-null     category
 1   ZONA      51 non-null     category
dtypes: category(2)
memory usage: 1.3 KB


# CLASIFICACIÓN ZONA A LOS DATOS DE TRÁFICO

## LIMPIEZA VARIABLES ESTACIÓN

En primer lugar, modifico las variables estacion de cada dataframe para normalizarlas y que así, tengan la misma codificación

In [57]:
df_total["_EST_CODE_"] = extract_digits_to_int(df_total["FEST"])
df_estaciones_trafico["_EST_CODE_"] = extract_digits_to_int(df_estaciones_trafico["ESTACION"])

#Elimino variable FEST
df_total = df_total.drop(columns=["FEST"])
df_estaciones_trafico = df_estaciones_trafico.drop(columns=["ESTACION"])

## CLASIFICACIÓN

Añado al dataframe df_total la variable "ZONA" la cual voy a clasificar apoyandome en el dataframe df_estaciones_trafico

In [58]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98900 entries, 0 to 116701
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   FDIA        98900 non-null  datetime64[ns]
 1   TOTAL_HOR   98900 non-null  int64         
 2   _EST_CODE_  98900 non-null  Int64         
dtypes: Int64(1), datetime64[ns](1), int64(1)
memory usage: 3.1 MB


In [59]:
df_estaciones_trafico.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51 entries, 0 to 59
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   ZONA        51 non-null     category
 1   _EST_CODE_  51 non-null     Int64   
dtypes: Int64(1), category(1)
memory usage: 1.3 KB


In [60]:
df_total = df_total.merge(df_estaciones_trafico[["_EST_CODE_", "ZONA"]], on="_EST_CODE_", how="left")

In [61]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98900 entries, 0 to 98899
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   FDIA        98900 non-null  datetime64[ns]
 1   TOTAL_HOR   98900 non-null  int64         
 2   _EST_CODE_  98900 non-null  Int64         
 3   ZONA        98900 non-null  category      
dtypes: Int64(1), category(1), datetime64[ns](1), int64(1)
memory usage: 2.5 MB


In [62]:
df_estaciones_trafico.head(20)

,ZONA,_EST_CODE_
0,Centro,1
1,Oeste,2
2,Este,3
3,Oeste,4
4,Sur,5
5,Noreste,6
6,Suroeste,7
9,Suroeste,10
11,Centro,12
12,Centro,13


In [63]:
df_total.head(20)

,FDIA,TOTAL_HOR,_EST_CODE_,ZONA
0,2019-11-01,38883,1,Centro
1,2019-11-01,33144,2,Oeste
2,2019-11-01,33512,3,Este
3,2019-11-01,20397,4,Oeste
4,2019-11-01,54960,5,Sur
5,2019-11-01,13793,6,Noreste
6,2019-11-01,74480,7,Suroeste
7,2019-11-01,8511,10,Suroeste
8,2019-11-01,30918,12,Centro
9,2019-11-01,36918,13,Centro


In [64]:
df_total.tail(20)

,FDIA,TOTAL_HOR,_EST_CODE_,ZONA
98880,2025-03-31,14889,38,Noroeste
98881,2025-03-31,57525,40,Norte
98882,2025-03-31,37318,41,Norte
98883,2025-03-31,17901,43,Suroeste
98884,2025-03-31,30981,44,Centro
98885,2025-03-31,27926,45,Centro
98886,2025-03-31,28775,46,Sureste
98887,2025-03-31,38788,48,Este
98888,2025-03-31,18575,49,Oeste
98889,2025-03-31,12727,50,Este


# LIMPIEZA DATAFRAME GENERADO

## ELIMINACIÓN VARIABLE "_EST_CODE_"

In [65]:
#Elimino variable _EST_CODE_
df_total = df_total.drop(columns=["_EST_CODE_"])

## AGRUPO VALORES DE TRÁFICO POR ZONAS

In [66]:
# Agrupo por fecha y zona sumando TOTAL_HOR
df_total = (
    df_total.groupby(["FDIA", "ZONA"], as_index=False, observed=True)["TOTAL_HOR"]
    .sum()
)

In [67]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17802 entries, 0 to 17801
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   FDIA       17802 non-null  datetime64[ns]
 1   ZONA       17802 non-null  category      
 2   TOTAL_HOR  17802 non-null  int64         
dtypes: category(1), datetime64[ns](1), int64(1)
memory usage: 296.0 KB


# COMPROBACIÓN

## EXISTENCIA DE TODOS LOS DÍAS

In [68]:
# Rango:
min_fecha = df_total["FDIA"].min()
max_fecha = df_total["FDIA"].max()
rango_completo = pd.date_range(start=min_fecha, end=max_fecha, freq="D")

faltan_fechas = rango_completo.difference(df_total["FDIA"].unique())

if len(faltan_fechas) == 0:
    print("✅ El dataframe contiene todos los días del rango.")
else:
    print("❌ Faltan fechas:", faltan_fechas)

✅ El dataframe contiene todos los días del rango.


## TODAS LAS ZONAS TIENEN UN VALOR EN TODOS LOS DIAS

In [69]:
# Saco el número de zonas totales
zonas = df_total["ZONA"].unique()
n_zonas = len(zonas)

# Cuento registros por día
conteo_por_dia = df_total.groupby("FDIA")["ZONA"].nunique() # Agrupo por día y cuento cuantas zonas hay por días.

# Días incompletos
dias_incompletos = conteo_por_dia[conteo_por_dia < n_zonas] # Me quedaré con los días a los caules les falte alguna zona

if dias_incompletos.empty:
    print("✅ Todos los días tienen datos para todas las zonas.")
else:
    print("❌ Hay días con zonas faltantes:")
    print(dias_incompletos)

✅ Todos los días tienen datos para todas las zonas.


In [70]:
df_total.head(30)

,FDIA,ZONA,TOTAL_HOR
0,2019-11-01,Centro,212165
1,2019-11-01,Este,104689
2,2019-11-01,Noreste,99748
3,2019-11-01,Noroeste,30557
4,2019-11-01,Norte,104926
5,2019-11-01,Oeste,143079
6,2019-11-01,Sur,225331
7,2019-11-01,Sureste,59873
8,2019-11-01,Suroeste,165001
9,2019-11-02,Centro,309458


In [71]:
df_total.tail(30)

,FDIA,ZONA,TOTAL_HOR
17772,2025-03-28,Sur,296166
17773,2025-03-28,Sureste,87656
17774,2025-03-28,Suroeste,173228
17775,2025-03-29,Centro,260843
17776,2025-03-29,Este,117289
17777,2025-03-29,Noreste,120488
17778,2025-03-29,Noroeste,33896
17779,2025-03-29,Norte,121008
17780,2025-03-29,Oeste,173194
17781,2025-03-29,Sur,251406


# IMPUTACION VALORES FUERA DE RANGO

Quiero que el dataframe de los datos de tráfico cuente con datos en los mismos rangos temporales que los tiene el dataframe de la calidad del aire, esto es desde el 01 de enero de 2010 hasta el 30 de junio de 2025, por tanto, procedo a la imputacion de los valores.

Para ello, hacemos un análisis de series temporales univariante de la variable "TOTAL_HOR"

## PASO 1: DIAGNÓSTICO INICIAL DEL DATAFRAME ORIGINAL

In [72]:
df_total.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17802 entries, 0 to 17801
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   FDIA       17802 non-null  datetime64[ns]
 1   ZONA       17802 non-null  category      
 2   TOTAL_HOR  17802 non-null  int64         
dtypes: category(1), datetime64[ns](1), int64(1)
memory usage: 296.0 KB


In [73]:
print("== Vista general del CSV ==")
print(f"Filas: {len(df_total):,} | Columnas: {len(df_total.columns)}")
print("Columnas:", list(df_total.columns))
print("\nTipos iniciales:")
print(df_total.dtypes)

== Vista general del CSV ==
Filas: 17,802 | Columnas: 3
Columnas: ['FDIA', 'ZONA', 'TOTAL_HOR']

Tipos iniciales:
FDIA         datetime64[ns]
ZONA               category
TOTAL_HOR             int64
dtype: object


In [74]:
df_total.describe(include="all")

,FDIA,ZONA,TOTAL_HOR
count,17802,17802,17802.000000
unique,NaN,9,NaN
top,NaN,Centro,NaN
freq,NaN,1978,NaN
mean,2022-07-16 12:00:00.000000256,NaN,146442.347152
min,2019-11-01 00:00:00,NaN,305.000000
25%,2021-03-09 00:00:00,NaN,82144.250000
50%,2022-07-16 12:00:00,NaN,150064.500000
75%,2023-11-23 00:00:00,NaN,187848.000000
max,2025-03-31 00:00:00,NaN,367147.000000


In [75]:
# Se comprueba que en el rango temporal del dataframe, no existe ningún día sin imputar valores de tráfico
min_date = df_total["FDIA"].min()
max_date = df_total["FDIA"].max()

# Fechas únicas ordenadas
unique_dates = df_total.drop_duplicates(subset=["FDIA"])["FDIA"].sort_values().reset_index(drop=True)

# Días esperados vs observados en el span
expected_days = (unique_dates.max() - unique_dates.min()).days + 1
actual_unique_days = unique_dates.nunique()
missing_days = expected_days - actual_unique_days
missing_ratio = missing_days / expected_days if expected_days > 0 else 0.0

print("\n== Resumen temporal ==")
print(f"Rango temporal: {min_date.date()} → {max_date.date()}")
print(f"Días únicos observados: {actual_unique_days:,} de {expected_days:,}")
print(f"Días faltantes en el span: {missing_days:,}  (ratio: {missing_ratio:.4f})")


== Resumen temporal ==
Rango temporal: 2019-11-01 → 2025-03-31
Días únicos observados: 1,978 de 1,978
Días faltantes en el span: 0  (ratio: 0.0000)


## PASO 2: SEPARAR DATAFRAMES POR ZONAS

Se crean distintos sub-dataframes para cada zona, de modo que cada zona tiene una instancia por día

In [76]:
DATE_COL = "FDIA"
VALUE_COL = "TOTAL_HOR"
CAT_COL = "ZONA"

dfs_por_zona = {}
resumen = []

for zona, g in df_total.groupby(CAT_COL, observed=True): # El bucle for zona, g in ...
# zona es el valor de la categoría de la variable "ZONA"
# g es el sub-dataframe con solo las filas de esa categoría.
    
    g = g[[DATE_COL, VALUE_COL]].set_index(DATE_COL).sort_index() # Convierte la columna de fecha en el índice del dataframe.
    # ordena el dataframe por el índice (la fecha).
    
    # Guardamos en el diccionario
    dfs_por_zona[zona] = g
        
    resumen.append({
        "ZONA": zona,
        "start": g.index.min(),
        "end": g.index.max(),
        "n_filas": len(g)
    })

# Resumen

df_resumen = pd.DataFrame(resumen).sort_values("ZONA").reset_index(drop=True)

print("== Resumen por ZONA ==")
print(df_resumen.to_string(index=False))
print(f"\nSe han creado {len(dfs_por_zona)} sub-dataframes")


== Resumen por ZONA ==
    ZONA      start        end  n_filas
  Centro 2019-11-01 2025-03-31     1978
    Este 2019-11-01 2025-03-31     1978
 Noreste 2019-11-01 2025-03-31     1978
Noroeste 2019-11-01 2025-03-31     1978
   Norte 2019-11-01 2025-03-31     1978
   Oeste 2019-11-01 2025-03-31     1978
     Sur 2019-11-01 2025-03-31     1978
 Sureste 2019-11-01 2025-03-31     1978
Suroeste 2019-11-01 2025-03-31     1978

Se han creado 9 sub-dataframes


## PASO 3: EXTENSIÓN TEMPORAL 2010-01-01 → 2025-06-30

Vamos a proceder a estimar los valores de tráfico desde el 01 de enero de 2010 hasta el 30 de junio de 2025 y que queden fuera del rango de fechas con los que ya cuento,esto es desde el 01 de noviembre de 2019 hasta el 31 de marzo de 2025.

Para la estimación llevaremos a cabo la siguiente estrategia:

1. Aplicación modelo determinista con **Series de Fourier**, ya que **SUPONEMOS** que la variable observada sigue algún tipo de esquema o patrón de comportamiento más o menos fijo.
   - Tendencia lineal + Fourier semanal + Fourier anual.
   - Pros: simple, rápido, transparente.
   - Contras: no modela shocks ni dinámicas residuales, riesgo de infraajuste si los patrones son complejos.

2. Aplicación **Modelo Prophet**
   - Modelo aditivo generalizado (GAM), que son una clase de modelos de regresión con suavizadores potencialmente no lineales aplicados a los regresores.
   - Tendencia con splines + Fourier para estacionalidad + capacidad de incluir festivos/efectos externos. 
   - Pros: combina lo mejor de Fourier y Splines (para la tendencia), robusto a outliers, flexible.
   - Contras: menos interpretable que Fourier puro, más pesado de entrenar.

3. Evaluación de riesgos (errores de predicción)
    - Nos va a servir para comparar ambos modelos y quedarnos con el que mejor prediga
    - Usamos cross-validation rolling-origin (entrenar hasta un punto, predecir un horizonte, calcular error, mover la ventana).
    - Métricas: RMSE, MAE.

4. Selección del modelo
   - Nos quedamos con el que tenga menor error y menor riesgo (menor varianza de errores).
   - Si están muy parejos, podemos considerar promediar ambos resultados.

In [77]:
# PASO 3.1: EXTENSIÓN TEMPORAL USANDO SERIES DE FOURIER:

# --------------------------------------------------------

START_TARGET = pd.Timestamp("2010-01-01")
END_TARGET   = pd.Timestamp("2025-06-30")
FULL_INDEX   = pd.date_range(START_TARGET, END_TARGET, freq="D")

extended_by_zone = {} # Diccionario para guardar las series extendidas (1 por zona)
panel_extended_FOURIER = []  # para construir un panel final apilado con todas las ZONAS

for zona, g in dfs_por_zona.items(): # Itera para cada zona y su correspondiente Dataframe g.
    # g: DataFrame con índice de fechas (DatetimeIndex) y columna VALUE_COL
    # Usamos el diccionario dfs_por_zona donde se encuentran todos los dataframes divididos por zonas
    if g.empty:
        continue

    # Asegurar tipos y orden
    g = g.copy().sort_index()
    if not isinstance(g.index, pd.DatetimeIndex):
        raise TypeError(f"El índice del sub-dataframe de la zona '{zona}' debe ser DatetimeIndex.") # Asegura que el índice esté ordenado y sea de tipo fecha (DatetimeIndex)
    y_obs = pd.to_numeric(g[VALUE_COL], errors="coerce").astype(float).values # Extrae los valores observados (TOTAL_HOR) como array numérico.
    dates_obs = g.index # Guarda las fechas correspondientes.

# APLICACIÓN SERIES DE FOURIER:

    # 1) Predicción determinista en TODO el rango objetivo
    y_pred_full = fit_predict_fourier(dates_train=dates_obs, y_train=y_obs, dates_pred=FULL_INDEX) # Predice valores para todo el rango objetivo (FULL_INDEX)

    # 2) Componer la serie extendida:
    #    - Fuera del tramo observado: usar la predicción
    #    - Dentro del tramo observado: conservar los valores observados
    ext = pd.Series(y_pred_full, index=FULL_INDEX, name=f"{VALUE_COL}_EXT") # Crea una serie con las predicciones.
    ext.loc[dates_obs.min():dates_obs.max()] = g[VALUE_COL].astype(float).values # Reemplaza los valores dentro del tramo observado con los datos reales.
    # 🔹 Convertir a enteros (redondeando primero)
    ext = ext.round().astype("Int64")

    # 3) Guardar por ZONA
    df_ext = ext.to_frame()
    df_ext.index.name = DATE_COL
    extended_by_zone[zona] = df_ext

    # 4) Añadir al panel combinado
    tmp = df_ext.copy()
    tmp[CAT_COL] = zona
    panel_extended_FOURIER.append(tmp.reset_index())


In [78]:
# PASO 3.2: EXTENSIÓN TEMPORAL USANDO MODELO PROPHET:

# --------------------------------------------------------

# --- Config reproducibilidad y logging ---
SEED = 42           # semilla de aleatoriedad
MCMC_SAMPLES = 0    # 0 = MAP (determinista, más rápido). >0 = MCMC (bayesiano)

np.random.seed(SEED) 
logging.getLogger("cmdstanpy").setLevel(logging.CRITICAL)  # silencia logs

extended_by_zone = {}
panel_extended_PROPHET = []

for zona, g in dfs_por_zona.items():
    if g.empty:
        continue

    g = g.copy().sort_index()
    if not isinstance(g.index, pd.DatetimeIndex):
        raise TypeError(f"El índice del sub-dataframe de la zona '{zona}' debe ser DatetimeIndex.")

    dates_obs = g.index
    y_obs = pd.to_numeric(g[VALUE_COL], errors="coerce").astype(float).values

    # 1) Predicción Prophet para todo el rango objetivo (con MCMC opcional + seed en fit)
    y_pred_full = prophet_fit_predict_full(dates_obs, y_obs, FULL_INDEX,
                                           mcmc_samples=MCMC_SAMPLES, seed=SEED)

    # 2) Componer serie extendida (observado dentro, predicho fuera)
    ext = y_pred_full.copy()
    ext.loc[dates_obs.min():dates_obs.max()] = g[VALUE_COL].astype(float).values

    # 3) Redondear a enteros
    ext = ext.round().astype("Int64")

    # 4) Guardar por ZONA
    df_ext = ext.to_frame()
    df_ext.index.name = DATE_COL
    extended_by_zone[zona] = df_ext

    # 5) Añadir al panel combinado
    tmp = df_ext.copy()
    tmp[CAT_COL] = zona
    panel_extended_PROPHET.append(tmp.reset_index())
    

13:32:55 - cmdstanpy - INFO - Chain [1] start processing
13:32:58 - cmdstanpy - INFO - Chain [1] done processing
13:33:00 - cmdstanpy - INFO - Chain [1] start processing
13:33:01 - cmdstanpy - INFO - Chain [1] done processing
13:33:03 - cmdstanpy - INFO - Chain [1] start processing
13:33:03 - cmdstanpy - INFO - Chain [1] done processing
13:33:05 - cmdstanpy - INFO - Chain [1] start processing
13:33:06 - cmdstanpy - INFO - Chain [1] done processing
13:33:07 - cmdstanpy - INFO - Chain [1] start processing
13:33:07 - cmdstanpy - INFO - Chain [1] done processing
13:33:09 - cmdstanpy - INFO - Chain [1] start processing
13:33:10 - cmdstanpy - INFO - Chain [1] done processing
13:33:12 - cmdstanpy - INFO - Chain [1] start processing
13:33:12 - cmdstanpy - INFO - Chain [1] done processing
13:33:14 - cmdstanpy - INFO - Chain [1] start processing
13:33:15 - cmdstanpy - INFO - Chain [1] done processing
13:33:17 - cmdstanpy - INFO - Chain [1] start processing
13:33:17 - cmdstanpy - INFO - Chain [1]

In [79]:
# PASO 3.3: ERRORES DE LAS PREDICCIONES:

# --------------------------------------------------------

# --- construir all_rows y summ_rows recorriendo las zonas ---
all_rows = []
summ_rows = []

for zona, g in dfs_por_zona.items():
    if g is None or g.empty:
        continue
    g = g.copy().sort_index()
    if not isinstance(g.index, pd.DatetimeIndex):
        raise TypeError(f"Zona '{zona}': el índice debe ser DatetimeIndex.")

    dates_obs = g.index
    y_obs = pd.to_numeric(g[VALUE_COL], errors="coerce").astype(float).values

    # CV rolling-origin para esta zona (usa tu rolling_cv_zone ya definida)
    cv_df = rolling_cv_zone(dates_obs, y_obs)
    if cv_df.empty:
        continue

    cv_df.insert(0, "ZONA", zona)
    all_rows.append(cv_df)

    # resumen por modelo para esta zona
    grp = cv_df.groupby("model", as_index=False).agg(
        rmse_mean=("rmse", "mean"),
        rmse_std=("rmse", "std"),
        mae_mean=("mae", "mean"),
        mae_std=("mae", "std"),
        n_splits=("rmse", "count")
    )
    grp.insert(0, "ZONA", zona)
    summ_rows.append(grp)

# --- combinar resultados ---
cv_results_all = pd.concat(all_rows, axis=0, ignore_index=True) if all_rows else pd.DataFrame()
cv_summary_by_zone = pd.concat(summ_rows, axis=0, ignore_index=True) if summ_rows else pd.DataFrame()

# --- elegir mejor modelo por zona ---
if not cv_summary_by_zone.empty:
    winners = (cv_summary_by_zone
               .groupby("ZONA", as_index=False)
               .apply(lambda g: pick_winner(g))
               .reset_index(drop=True))
else:
    winners = pd.DataFrame(columns=["ZONA","best_model_by_rmse","rmse_mean","mae_mean","n_splits","ensemble_suggested"])

# --- imprimir ---
print("=== CV COMPLETADO ===")
print(f"Zonas evaluadas: {len(dfs_por_zona)}")
print(f"Splits por zona (máx): {0 if cv_results_all.empty else cv_results_all['split'].nunique()}")
if not cv_summary_by_zone.empty:
    print("\n> Resumen por zona y modelo:")
    print(cv_summary_by_zone.to_string(index=False))
    print("\n> Mejor modelo por zona (con sugerencia de ensemble si RMSE ~empata):")
    print(winners.to_string(index=False))
else:
    print("No hay resultados de CV (revisa que rolling_cv_zone devuelva filas).")


13:33:19 - cmdstanpy - INFO - Chain [1] start processing
13:33:19 - cmdstanpy - INFO - Chain [1] done processing
13:33:19 - cmdstanpy - INFO - Chain [1] start processing
13:33:20 - cmdstanpy - INFO - Chain [1] done processing
13:33:20 - cmdstanpy - INFO - Chain [1] start processing
13:33:20 - cmdstanpy - INFO - Chain [1] done processing
13:33:20 - cmdstanpy - INFO - Chain [1] start processing
13:33:20 - cmdstanpy - INFO - Chain [1] done processing
13:33:21 - cmdstanpy - INFO - Chain [1] start processing
13:33:21 - cmdstanpy - INFO - Chain [1] done processing
13:33:21 - cmdstanpy - INFO - Chain [1] start processing
13:33:21 - cmdstanpy - INFO - Chain [1] done processing
13:33:21 - cmdstanpy - INFO - Chain [1] start processing
13:33:22 - cmdstanpy - INFO - Chain [1] done processing
13:33:22 - cmdstanpy - INFO - Chain [1] start processing
13:33:22 - cmdstanpy - INFO - Chain [1] done processing
13:33:22 - cmdstanpy - INFO - Chain [1] start processing
13:33:23 - cmdstanpy - INFO - Chain [1]

=== CV COMPLETADO ===
Zonas evaluadas: 9
Splits por zona (máx): 20

> Resumen por zona y modelo:
    ZONA   model    rmse_mean     rmse_std     mae_mean      mae_std  n_splits
  Centro fourier 50518.856671 16229.480048 41957.177598 13711.204973        20
  Centro prophet 33455.535658 17012.441500 28953.537469 16846.918460        20
    Este fourier 43212.046902 14674.312007 34927.008479 13391.546703        20
    Este prophet 20000.240204 11221.070167 15710.281162  9441.895560        20
 Noreste fourier 44283.334997 17365.234550 34917.233316 16686.642856        20
 Noreste prophet 22079.674774 12304.734692 18139.350891 11430.070089        20
Noroeste fourier 15410.681566  5919.477926 12380.194897  5758.133328        20
Noroeste prophet  7697.741284  3532.206608  6465.379061  3214.426124        20
   Norte fourier 35806.768660 13837.108649 28772.560478 12563.008878        20
   Norte prophet 19082.519124  9802.170936 15325.280215  8628.617377        20
   Oeste fourier 32195.276681 1349

C:\Users\rosal\AppData\Local\Temp\ipykernel_52296\3793801252.py:46: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pick_winner(g))


De los resultado anteriores, concluimos que debemos tomar los datos predichos con el modelo Prophet, por tanto:

In [80]:
# PASO 3.4: SELECCIÓN DEL MODELO

datos_tráfico_def = pd.concat(panel_extended_PROPHET, axis=0, ignore_index=True)

In [81]:
datos_tráfico_def.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50940 entries, 0 to 50939
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   FDIA           50940 non-null  datetime64[ns]
 1   TOTAL_HOR_EXT  50940 non-null  Int64         
 2   ZONA           50940 non-null  object        
dtypes: Int64(1), datetime64[ns](1), object(1)
memory usage: 1.2+ MB


In [82]:
datos_tráfico_def["ZONA"] = datos_tráfico_def["ZONA"].astype("category")
datos_tráfico_def = datos_tráfico_def.sort_values("FDIA").reset_index(drop=True)
datos_tráfico_def.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50940 entries, 0 to 50939
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   FDIA           50940 non-null  datetime64[ns]
 1   TOTAL_HOR_EXT  50940 non-null  Int64         
 2   ZONA           50940 non-null  category      
dtypes: Int64(1), category(1), datetime64[ns](1)
memory usage: 895.9 KB


In [83]:
datos_tráfico_def.describe(include="all")

,FDIA,TOTAL_HOR_EXT,ZONA
count,50940,50940.0,50940
unique,NaN,<NA>,9
top,NaN,<NA>,Centro
freq,NaN,<NA>,5660
mean,2017-09-30 12:00:00,714785.612662,NaN
min,2010-01-01 00:00:00,305.0,NaN
25%,2013-11-15 18:00:00,167512.75,NaN
50%,2017-09-30 12:00:00,403614.5,NaN
75%,2021-08-15 06:00:00,1176631.5,NaN
max,2025-06-30 00:00:00,3066700.0,NaN


In [84]:
datos_tráfico_def.isna().sum()

FDIA             0
TOTAL_HOR_EXT    0
ZONA             0
dtype: int64

In [85]:
datos_tráfico_def.head(30)

,FDIA,TOTAL_HOR_EXT,ZONA
0,2010-01-01,2937562,Centro
1,2010-01-01,2068629,Norte
2,2010-01-01,1838364,Oeste
3,2010-01-01,2244516,Suroeste
4,2010-01-01,1812548,Este
5,2010-01-01,746307,Sureste
6,2010-01-01,3047404,Sur
7,2010-01-01,1872829,Noreste
8,2010-01-01,528176,Noroeste
9,2010-01-02,2988706,Sur


In [86]:
datos_tráfico_def.tail(30)

,FDIA,TOTAL_HOR_EXT,ZONA
50910,2025-06-27,209156,Oeste
50911,2025-06-27,165287,Noreste
50912,2025-06-27,269267,Sur
50913,2025-06-28,214409,Sur
50914,2025-06-28,118996,Este
50915,2025-06-28,62620,Sureste
50916,2025-06-28,34661,Noroeste
50917,2025-06-28,105180,Norte
50918,2025-06-28,114476,Noreste
50919,2025-06-28,175420,Oeste


In [87]:
datos_tráfico_def.sample(30)

,FDIA,TOTAL_HOR_EXT,ZONA
65,2010-01-08,2928074,Centro
26241,2017-12-25,535317,Norte
41605,2022-08-28,43806,Sureste
285,2010-02-01,1877647,Noreste
31100,2019-06-18,275426,Norte
46955,2024-04-14,55286,Sureste
49554,2025-01-28,158306,Noreste
12473,2013-10-17,1212058,Este
12655,2013-11-07,1939102,Centro
33567,2020-03-18,28620,Sureste


In [88]:
datos_tráfico_def.to_parquet("03_DATASET_FINAL.parquet", engine="pyarrow", index=False)